In [79]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [80]:
bd.projects.set_current('ei38-teaching-25')
ipcc=('IPCC 2013', 'climate change', 'GWP 100a')

In [81]:
ei = bd.Database('ei 3.8 cutoff')
activity= ei.get('6a7848cd768f5163834eec59c80de49b')
ei_pack=ei.datapackage()

In [171]:
lca1 = bc.LCA({activity: 1}, ipcc)
lca1.lci()
lca1.lcia()
lca1.score

2.477976880419227

In [82]:
#[act for act in ei if 'market for acrylic acid' in  act['name']][1].id

In [83]:
heat_acts=[act for act in ei if 'heat production' in  act['name']
#and ('RER' in act['location'] or 'DE' in act['location'] or 'Europe without Switzerland' in act['location'])
#and 'heat' in act['reference product'] 
and 'heat pump' not in act['name']
and 'biomethane' not in act['name']
#and ('natural gas' in act['name'] or 'anthracite' in act['name'])
]

In [100]:
new_heat_supply=[act for act in ei if 'market for electricity, medium voltage' in act['name'] and 'DE' in act['location']][0]
new_heat_supply

'electricity, from municipal waste incineration to generic market for electricity, medium voltage' (kilowatt hour, DE, None)

In [84]:
exchange =list(activity.technosphere())[0]
a= np.array((exchange.input.id, exchange.output.id), bwp.INDICES_DTYPE)

In [85]:
np.where(ei_pack.data[5] == a)

(array([157541], dtype=int64),)

In [86]:
index=[]
flow_list=[]
global depth
depth=0
max_depth=2

In [87]:
ei_pack.data[5][3]['row']

4428

In [88]:
def recurse_supply(act, depth=0, max_depth=1, flow_dic={},amount=1):
    #global depth
    for exc in act.technosphere():
        
        if exc.input in heat_acts:
            amount = exc['amount']*amount
            flow_dic[exc.input]=amount
            
        elif depth< max_depth:
            depth +=1 
            depth, flow_dic = recurse_supply(exc.input,depth, max_depth, flow_dic, amount)
    
    depth -= 1
    return depth, flow_dic

In [89]:
def recurse_supply(act, depth=0, max_depth=1, flow_dic={},amount=1):
    #global depth
    for exc in act.technosphere():
        
        if exc.input in heat_acts:
            amount = exc['amount']*amount
            flow_dic[exc.input]=amount
            
        elif depth< max_depth:
            depth +=1 
            depth, flow_dic = recurse_supply(exc.input,depth, max_depth, flow_dic, amount)
    
    depth -= 1
    return depth, flow_dic

In [134]:
def recurse_supply(act):
    global depth
    for exc in act.technosphere():
        if exc.input in heat_acts:
            #flow_list.append(np.array((exc.input.id, exc.output.id), bwp.INDICES_DTYPE))
            flow_list.append((exc.input.id, exc.output.id))
            #print(exc.input)
        elif depth< max_depth:
            depth +=1 
            recurse_supply(exc.input)
    depth -= 1
        

In [133]:
flow_list=[]
global depth
depth=0
max_depth=3
recurse_supply(activity)

'heat production, natural gas, at boiler modulating >100kW' (megajoule, Europe without Switzerland, None)
'heat production, hardwood chips from forest, at furnace 50kW, state-of-the-art 2014' (megajoule, CH, None)
'heat production, hardwood chips from forest, at furnace 50kW' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 100kW condensing, non-modulating' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 100kW, non-modulating' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 10kW condensing, non-modulating' (megajoule, CH, None)
'heat production, light fuel oil, at boiler 10kW, non-modulating' (megajoule, CH, None)
'heat production, mixed logs, at furnace 100kW, state-of-the-art 2014' (megajoule, CH, None)
'heat production, mixed logs, at furnace 100kW' (megajoule, CH, None)
'heat production, mixed logs, at furnace 30kW, state-of-the-art 2014' (megajoule, CH, None)
'heat production, mixed logs, at furnace 30kW' (megajoule, CH, None)
'

In [136]:
new_flow_list=[]
for exc in flow_list:
    new_flow_list.append((new_heat_supply.id, exc[1]))
    #new_flow_list.append(np.array((new_heat_supply.id, exc['col']), bwp.INDICES_DTYPE))


In [137]:
empty_data= np.zeros(len(flow_list))

In [ ]:
#new_flow_list.append(np.array((new_heat_supply.id, exc['col']), bwp.INDICES_DTYPE))

In [140]:
data_list=[]
for exc in flow_list:
    data_list.append(ei_pack.data[6][np.where(ei_pack.data[5] == np.array((exc), bwp.INDICES_DTYPE))][0])

In [165]:
indices = np.array(flow_list + new_flow_list, bwp.INDICES_DTYPE)
data = np.array(list(empty_data) + data_list)

In [167]:
dp = bwp.create_datapackage()

In [168]:

dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    name="new_heat_supply",
)

In [169]:
_, data_objs, _ = bd.prepare_lca_inputs({activity: 1}, ipcc)

In [172]:
lca2 = bc.LCA({activity.id: 1}, data_objs=data_objs + [dp])
lca2.lci()
lca2.lcia()

In [ ]:

print(lca1.score, lca2.score)